### Benchmark Calibration

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Import calibration methods (updated for v0.4.1)
from calibre import (
    IsotonicCalibrator,
    NearlyIsotonicCalibrator,
    RegularizedIsotonicCalibrator,
    RelaxedPAVACalibrator,
    SmoothedIsotonicCalibrator,
    SplineCalibrator,
)

# Import metrics (updated for v0.4.1)
from calibre.metrics import (
    binned_calibration_error,
    correlation_metrics,
    mean_calibration_error,
    unique_value_counts,
)


def generate_nonlinear_data(n_samples=1000, noise_level=0.1):
    """
    Generate synthetic data with non-linear true function and added noise.

    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    noise_level : float
        Standard deviation of the added Gaussian noise

    Returns:
    --------
    x : array
        Input features
    y_true : array
        True underlying function values
    y_noisy : array
        Noisy predictions
    """
    # Generate x values
    x = np.linspace(0, 1, n_samples)

    # Non-linear true function (e.g., a combination of sine and exponential)
    y_true = np.sin(2 * np.pi * x) * np.exp(x - 0.5)

    # Add Gaussian noise
    y_noisy = y_true + np.random.normal(0, noise_level, size=n_samples)

    return x, y_true, y_noisy


def benchmark_calibration_methods(noise_levels=[0.05, 0.1, 0.2]):
    """
    Benchmark different calibration methods across various noise levels.

    Parameters:
    -----------
    noise_levels : list
        Different noise levels to test

    Returns:
    --------
    results_df : pandas.DataFrame
        Comprehensive results of calibration methods
    """
    # Initialize results storage
    results = []

    # Iterate through different noise levels
    for noise in noise_levels:
        # Generate data
        x, y_true, y_noisy = generate_nonlinear_data(noise_level=noise)

        # Split data
        x_train, x_test, y_train, y_test, y_true_train, y_true_test = train_test_split(
            x, y_noisy, y_true, test_size=0.2, random_state=42
        )

        # Calibration methods to test (updated for v0.4.1)
        calibration_methods = [
            ("Vanilla Isotonic", IsotonicCalibrator()),
            ("Nearly Isotonic (Strict)", NearlyIsotonicCalibrator(lam=10.0)),
            ("Nearly Isotonic (Relaxed)", NearlyIsotonicCalibrator(lam=0.1)),
            ("I-Spline Calib", SplineCalibrator(n_splines=10, degree=3, cv=3)),
            ("Relax PAVA (10%)", RelaxedPAVACalibrator(percentile=10)),
            ("Regularized Isotonic", RegularizedIsotonicCalibrator(alpha=0.1)),
            (
                "Smoothed Isotonic",
                SmoothedIsotonicCalibrator(window_length=7, poly_order=3),
            ),
        ]

        # Compute metrics for each method
        for method_name, calibrator in calibration_methods:
            try:
                # Fit and transform using class-based API
                calibrator.fit(x_train, y_train)
                y_calibrated = calibrator.transform(x_train)

                # Evaluate metrics
                mce = mean_calibration_error(y_true_train, y_calibrated)
                bce = binned_calibration_error(y_true_train, y_calibrated, n_bins=10)
                corr = correlation_metrics(
                    y_true_train, y_calibrated, x=x_train, y_orig=y_train
                )
                unique_counts = unique_value_counts(y_calibrated, y_orig=y_train)

                # Store results
                results.append(
                    {
                        "Noise Level": noise,
                        "Method": method_name,
                        "Mean Calibration Error": mce,
                        "Binned Calibration Error": bce,
                        "Spearman Corr (True)": corr["spearman_corr_to_y_true"],
                        "Spearman Corr (Orig)": corr.get(
                            "spearman_corr_to_y_orig", np.nan
                        ),
                        "Unique Values Ratio": unique_counts["unique_value_ratio"],
                        "N Unique Values": unique_counts["n_unique_y_pred"],
                        "N Unique Values (Orig)": unique_counts["n_unique_y_orig"],
                    }
                )
            except Exception as e:
                print(f"Warning: {method_name} failed with error: {e}")

    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Run benchmark
benchmark_results = benchmark_calibration_methods()

# Display results
print(benchmark_results)

# Optional: Visualization of results
plt.figure(figsize=(15, 10))

# Metrics to plot
metrics_to_plot = [
    ("Mean Calibration Error", "Mean Calibration Error"),
    ("Unique Values Ratio", "Unique Values Ratio"),
    ("Spearman Corr (True)", "Spearman Correlation (True)"),
]

# Create subplots for each metric
for i, (col_name, title) in enumerate(metrics_to_plot, 1):
    plt.subplot(1, 3, i)
    for method in benchmark_results["Method"].unique():
        subset = benchmark_results[benchmark_results["Method"] == method]
        plt.plot(subset["Noise Level"], subset[col_name], marker="o", label=method)
    plt.title(title)
    plt.xlabel("Noise Level")
    plt.ylabel(col_name)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Import calibration methods (updated for v0.4.1)

# Import metrics (updated for v0.4.1)


def generate_continuous_data(n_samples=1000, noise_level=0.1):
    """
    Generate synthetic continuous data with non-linear true function and added noise.

    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    noise_level : float
        Standard deviation of the added Gaussian noise

    Returns:
    --------
    x : array
        Input features
    y_true : array
        True underlying function values
    y_noisy : array
        Noisy predictions
    """
    # Multiple non-linear functions to create complexity
    np.random.seed(42)

    # Generate x values with more complex distribution
    x = np.random.uniform(0, 1, n_samples)

    # Combine multiple non-linear transformations
    y_true = (
        np.sin(2 * np.pi * x) * np.exp(x - 0.5)  # Sine with exponential
        + 0.5 * x**2  # Quadratic component
        + 0.3 * np.log(x + 0.1)  # Logarithmic component
    )

    # Standardize to control overall scale
    scaler = StandardScaler()
    y_true = scaler.fit_transform(y_true.reshape(-1, 1)).ravel()

    # Add Gaussian noise
    y_noisy = y_true + np.random.normal(0, noise_level, size=n_samples)

    return x, y_true, y_noisy


def benchmark_calibration_methods(noise_levels=[0.05, 0.1, 0.2]):
    """
    Benchmark different calibration methods across various noise levels.

    Parameters:
    -----------
    noise_levels : list
        Different noise levels to test

    Returns:
    --------
    results_df : pandas.DataFrame
        Comprehensive results of calibration methods
    """
    # Initialize results storage
    results = []

    # Iterate through different noise levels
    for noise in noise_levels:
        # Generate data
        x, y_true, y_noisy = generate_continuous_data(noise_level=noise)

        # Split data
        x_train, x_test, y_train, y_test, y_true_train, y_true_test = train_test_split(
            x, y_noisy, y_true, test_size=0.2, random_state=42
        )

        # Calibration methods to test (updated for v0.4.1)
        calibration_methods = [
            ("Vanilla Isotonic", IsotonicCalibrator()),
            ("Nearly Isotonic (Strict)", NearlyIsotonicCalibrator(lam=10.0)),
            ("Nearly Isotonic (Relaxed)", NearlyIsotonicCalibrator(lam=0.1)),
            ("I-Spline Calib", SplineCalibrator(n_splines=10, degree=3, cv=3)),
            ("Relax PAVA (10%)", RelaxedPAVACalibrator(percentile=10)),
            ("Regularized Isotonic", RegularizedIsotonicCalibrator(alpha=0.1)),
            (
                "Smoothed Isotonic",
                SmoothedIsotonicCalibrator(window_length=7, poly_order=3),
            ),
        ]

        # Compute metrics for each method
        for method_name, calibrator in calibration_methods:
            try:
                # Fit and transform using class-based API
                calibrator.fit(x_train, y_train)
                y_calibrated = calibrator.transform(x_train)

                # Evaluate metrics
                mce = mean_calibration_error(y_true_train, y_calibrated)
                bce = binned_calibration_error(y_true_train, y_calibrated, n_bins=10)
                corr = correlation_metrics(
                    y_true_train, y_calibrated, x=x_train, y_orig=y_train
                )
                unique_counts = unique_value_counts(y_calibrated, y_orig=y_train)

                # Store results
                results.append(
                    {
                        "Noise Level": noise,
                        "Method": method_name,
                        "Mean Calibration Error": mce,
                        "Binned Calibration Error": bce,
                        "Spearman Corr (True)": corr["spearman_corr_to_y_true"],
                        "Spearman Corr (Orig)": corr.get(
                            "spearman_corr_to_y_orig", np.nan
                        ),
                        "Unique Values Ratio": unique_counts["unique_value_ratio"],
                        "N Unique Values": unique_counts["n_unique_y_pred"],
                        "N Unique Values (Orig)": unique_counts["n_unique_y_orig"],
                    }
                )
            except Exception as e:
                print(f"Warning: {method_name} failed with error: {e}")

    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Run benchmark
benchmark_results = benchmark_calibration_methods()

# Display results
print(benchmark_results)

# Visualization
plt.figure(figsize=(16, 5))

# Metrics to plot
metrics_to_plot = [
    ("Mean Calibration Error", "Mean Calibration Error"),
    ("Unique Values Ratio", "Unique Values Preservation"),
    ("Spearman Corr (True)", "Correlation with True Values"),
]

# Create subplots for each metric
for i, (col_name, title) in enumerate(metrics_to_plot, 1):
    plt.subplot(1, 3, i)
    for method in benchmark_results["Method"].unique():
        subset = benchmark_results[benchmark_results["Method"] == method]
        plt.plot(subset["Noise Level"], subset[col_name], marker="o", label=method)
    plt.title(title)
    plt.xlabel("Noise Level")
    plt.ylabel(col_name)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

plt.tight_layout()
plt.show()